In [4]:
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Data augmentation
import os
import shutil
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score

# Define dataset paths
base_path = '/content/drive/My Drive/Colab Notebooks/MP'
image_dir = os.path.join(base_path, 'images')
label_dir = os.path.join(base_path, 'labels')

print("Number of original images:",len(os.listdir(image_dir)))
print("Number of original labels:",len(os.listdir(label_dir)))

# Define new paths for training and validation splits
train_image_dir = os.path.join(image_dir, 'train')
val_image_dir = os.path.join(image_dir, 'val')
train_label_dir = os.path.join(label_dir, 'train')
val_label_dir = os.path.join(label_dir, 'val')

# Create directories if they don't exist
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

def rotate_image_and_labels(image, labels, angle):
    """Rotate image and its corresponding labels."""
    (h, w) = image.shape[:2]
    rotated_labels = []
    if angle == 90:
        rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        for label in labels:
            cls, x, y, width, height = label
            new_x = 1 - y
            new_y = x
            rotated_labels.append((cls, new_x, new_y, height, width))
    elif angle == 180:
        rotated_image = cv2.rotate(image, cv2.ROTATE_180)
        for label in labels:
            cls, x, y, width, height = label
            new_x = 1 - x
            new_y = 1 - y
            rotated_labels.append((cls, new_x, new_y, width, height))
    elif angle == 270:
        rotated_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        for label in labels:
            cls, x, y, width, height = label
            new_x = y
            new_y = 1 - x
            rotated_labels.append((cls, new_x, new_y, height, width))
    else:
        raise ValueError("Unsupported rotation angle")

    return rotated_image, rotated_labels

def augment_data(image_path, label_path, output_image_dir, output_label_dir):
    """Augment data by rotating images and labels."""
    image = cv2.imread(image_path)
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            label = list(map(float, line.strip().split()))
            labels.append(label)

    angles = [90, 180, 270]
    for angle in angles:
        rotated_image, rotated_labels = rotate_image_and_labels(image, labels, angle)
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        new_image_path = os.path.join(output_image_dir, f"{base_name}_rot{angle}.jpg")
        new_label_path = os.path.join(output_label_dir, f"{base_name}_rot{angle}.txt")
        cv2.imwrite(new_image_path, rotated_image)
        with open(new_label_path, 'w') as file:
            for label in rotated_labels:
                file.write(" ".join(map(str, label)) + "\n")
# Set the split ratio
split_ratio = 0.8  # 80% training, 20% validation

# Get list of all images and corresponding labels
all_images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
all_labels = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# Match images with corresponding labels
matched_pairs = [(img, img.replace('.jpg', '.txt')) for img in all_images if img.replace('.jpg', '.txt') in all_labels]

# Debug: Print out the matched pairs
print("Matched pairs:", matched_pairs)

# Check if the lists are correctly populated
if not matched_pairs:
    raise ValueError("No matching images and labels found. Please check the paths and file extensions.")

# Shuffle the dataset
random.shuffle(matched_pairs)

# Split the dataset
split_index = int(len(matched_pairs) * split_ratio)
train_pairs = matched_pairs[:split_index]
val_pairs = matched_pairs[split_index:]

# Copy and augment training data
for img, lbl in train_pairs:
    shutil.copy(os.path.join(image_dir, img), train_image_dir)
    shutil.copy(os.path.join(label_dir, lbl), train_label_dir)
    augment_data(os.path.join(image_dir, img), os.path.join(label_dir, lbl), train_image_dir, train_label_dir)

# Copy and augment validation data
for img, lbl in val_pairs:
    shutil.copy(os.path.join(image_dir, img), val_image_dir)
    shutil.copy(os.path.join(label_dir, lbl), val_label_dir)
    augment_data(os.path.join(image_dir, img), os.path.join(label_dir, lbl), val_image_dir, val_label_dir)

# Verify the split
print("Training images:", len(os.listdir(train_image_dir)))
print("Training labels:", len(os.listdir(train_label_dir)))
print("Validation images:", len(os.listdir(val_image_dir)))
print("Validation labels:", len(os.listdir(val_label_dir)))

Number of original images: 1382
Number of original labels: 516
Matched pairs: [('5_383.jpg', '5_383.txt'), ('5_384.jpg', '5_384.txt'), ('5_386.jpg', '5_386.txt'), ('5_391.jpg', '5_391.txt'), ('5_390.jpg', '5_390.txt'), ('5_394.jpg', '5_394.txt'), ('5_395.jpg', '5_395.txt'), ('5_397.jpg', '5_397.txt'), ('5_398.jpg', '5_398.txt'), ('5_403.jpg', '5_403.txt'), ('5_406.jpg', '5_406.txt'), ('5_410.jpg', '5_410.txt'), ('5_415.jpg', '5_415.txt'), ('5_428.jpg', '5_428.txt'), ('5_436.jpg', '5_436.txt'), ('5_438.jpg', '5_438.txt'), ('5_441.jpg', '5_441.txt'), ('5_442.jpg', '5_442.txt'), ('5_443.jpg', '5_443.txt'), ('5_448.jpg', '5_448.txt'), ('5_449.jpg', '5_449.txt'), ('5_450.jpg', '5_450.txt'), ('5_451.jpg', '5_451.txt'), ('5_453.jpg', '5_453.txt'), ('5_458.jpg', '5_458.txt'), ('5_459.jpg', '5_459.txt'), ('5_464.jpg', '5_464.txt'), ('5_468.jpg', '5_468.txt'), ('5_470.jpg', '5_470.txt'), ('5_471.jpg', '5_471.txt'), ('5_474.jpg', '5_474.txt'), ('5_479.jpg', '5_479.txt'), ('5_480.jpg', '5_480.txt'

In [16]:
data_yaml_content = """
train: /content/drive/My Drive/Colab Notebooks/MP/images/train
val: /content/drive/My Drive/Colab Notebooks/MP/images/val

nc: 4  # number of classes
names: [0, 5, 10, 30]  # class names
"""

# Save the data.yaml file to the appropriate directory
with open('/content/drive/My Drive/Colab Notebooks/MP/data.yaml', 'w') as file:
    file.write(data_yaml_content)

In [17]:
model = YOLO("yolov8n.yaml")  # build a new model from YAML
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolov8n.yaml").load("yolov8n.pt")  # build from YAML and transfer weights

data="/content/drive/My Drive/Colab Notebooks/MP/data.yaml"
epochs=200
imgsz=640
patience=10
plots=True
save_period = 1
project = "/content/drive/My Drive/Colab Notebooks/MP/training_results"
verbose = True
# Train the model
results = model.train(data=data, epochs=epochs, imgsz=imgsz, patience=patience, plots=plots, save_period=save_period, project=project, verbose=verbose)

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/My Drive/Colab Notebooks/MP/data.yaml, epochs=200, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=/content/drive/My Drive/Colab Notebooks/MP/training_results, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False

train: Scanning /content/drive/My Drive/Colab Notebooks/MP/labels/train... 2016 images, 8 backgrounds, 0 corrupt: 100%|██████████| 2016/2016 [00:25<00:00, 77.65it/s] 


train: New cache created: /content/drive/My Drive/Colab Notebooks/MP/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/Colab Notebooks/MP/labels/val... 748 images, 0 backgrounds, 0 corrupt: 100%|██████████| 748/748 [00:05<00:00, 140.44it/s]


val: New cache created: /content/drive/My Drive/Colab Notebooks/MP/labels/val.cache
Plotting labels to /content/drive/My Drive/Colab Notebooks/MP/training_results/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/Colab Notebooks/MP/training_results/train2
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     19/200      2.28G      1.469     0.6664      1.155         51        640: 100%|██████████| 126/126 [00:13<00:00,  9.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:03<00:00,  6.62it/s]

                   all        748       1516       0.79      0.755      0.769      0.414
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



19 epochs completed in 0.107 hours.
Optimizer stripped from /content/drive/My Drive/Colab Notebooks/MP/training_results/train2/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/My Drive/Colab Notebooks/MP/training_results/train2/weights/best.pt, 6.2MB

Validating /content/drive/My Drive/Colab Notebooks/MP/training_results/train2/weights/best.pt...
Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8n summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:05<00:00,  4.23it/s]


                   all        748       1516      0.875      0.906       0.93      0.509
                     0        264        328      0.957      0.951      0.969      0.381
                     5        432       1076      0.907      0.981      0.983      0.615
                    10          8         12       0.95          1      0.995      0.735
                    30         88        100      0.686       0.69      0.774      0.307
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /content/drive/My Drive/Colab Notebooks/MP/training_results/train2
